## Variables are labels not boxes

In [ ]:
a = [1,2,34]
b = a   
# a nd b hold reference to same list , 
# a = b does not create copy the contents of a to b. 
# it just attach label b to the object that already has label a
b.append(5)

a, b

([1, 2, 34, 5], [1, 2, 34, 5])

### variables are bound to object only after the object are created

In [2]:
class Gizmo:
    def __init__(self):
        print(f"Gizmo id : {id(self)}")

g = Gizmo()

Gizmo id : 4376645312


In [3]:
y = Gizmo() * 10

Gizmo id : 4376195984


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

### Identity, Equality and Aliases

In [4]:
charles = {"name": "Charles L. Doggson", "born": 1832}
lewis = charles

lewis is charles

True

In [5]:
id(charles), id(lewis)

(4568845632, 4568845632)

In [6]:
lewis['balance'] = 950
charles

{'name': 'Charles L. Doggson', 'born': 1832, 'balance': 950}

In [8]:
alex = {'name': 'Charles L. Doggson', 'born': 1832, 'balance': 950}

alex == charles, alex is charles

(True, False)

### Relative Immutability of Tuples

In [10]:
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])

t1 == t2, t1 is t2

(True, False)

In [12]:
id(t1[-1]), id(t2[-1])

(4569046976, 4569048192)

In [13]:
t1[-1].append(99)
t1

(1, 2, [30, 40, 99])

In [14]:
t1 == t2

False

### Copies are Shallow by Default

In [25]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)  # create shallow copy of l1, the outermost container is duplicated
l2

[3, [55, 44], (7, 8, 9)]

In [23]:
l3 = l1[:]   # this also produces shallow copy
l3

[3, [55, 44], (7, 8, 9)]

In [ ]:
l1.append(100)
l1, l2

([3, [55, 44], (7, 8, 9), 100], [3, [55, 44], (7, 8, 9)])

In [ ]:
l2[1] += [33, 44]

# changes in l2 affected l1 also because l2 is shallow copy of l1
l1, l2

([3, [55, 44, 33, 44], (7, 8, 9)], [3, [55, 44, 33, 44], (7, 8, 9)])

### Deep and Shalow copies of Arbitrary Objects

In [60]:
class Bus:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
    
    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [31]:
import copy
bus1 = Bus(['Mini', 'Coco', 'Dora', 'Oreo'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
id(bus1), id(bus2), id(bus3)

(4376652704, 4568758416, 4376194704)

In [32]:
bus1.drop('Dora')
bus2.passengers

['Mini', 'Coco', 'Oreo']

In [33]:
bus3.passengers

['Mini', 'Coco', 'Dora', 'Oreo']

In [35]:
a = [10, 20]
b = [a, 30]
a.append(b)
a

[10, 20, [[...], 30]]

In [36]:
c = copy.deepcopy(a)
c

[10, 20, [[...], 30]]

#### one can control the behaviour of copy and deepcopy by implelemting
<pre> __copy__ and __deepcopy__</pre>

In [ ]:
class Foo:
    def __init__(self, x, data):
        self.x = x
        self.data = data
        self.cached = {'expensive': 42}
    
    def __copy__(self):
        new = type(self)(self.x, self.data)
        new.cached = self.cached     # reusing cache
        return new
    
    def __deep_copy__(self, memo):
        # necessary for deepcopy to not loop forever
        # memo is a dict created internally by copy.deepcopy. keeping a memo dictionary of objects already copied during the current copying pass
        if id(self) in memo:     
            return memo[id(self)]
        
        new = type(self)(
            copy.deepcopy(self.x, memo), 
            copy.deepcopy(self.data, memo)
        )

        new.cached = self.cached  # resuing cache

        memo[id(self)] = new

        return new

In [ ]:

f1 = Foo(10, [1,2,3])
f2 = copy.copy(f1)
f3 = copy.deepcopy(f1)

In [42]:
f1.data.append(99)
f2.data, f3.data

([1, 2, 3, 99], [1, 2, 3])

### Function Parameters as references

In [ ]:
def f(a, b):
    a += b
    return a

x, y = 1, 2
x,y

(1, 2)

In [45]:
print(f(x, y))
print(x,y)

3
1 2


In [46]:
a, b = [1, 2], [3, 4]
f(a, b)

[1, 2, 3, 4]

In [48]:
print(a)
print(b)

[1, 2, 3, 4]
[3, 4]


In [57]:
# fix
def f1(a, b):
    c = copy.copy(a)
    c += b
    return c

In [58]:
a, b = [1, 2], [3, 4]
print(f1(a, b))
print(a, b)

[1, 2, 3, 4]
[1, 2] [3, 4]


### Mutable Types as Parameter Defaults: Bad Idea

In [70]:
class HauntedBus:
    def __init__(self, passengers=[]):
        self.passengers = passengers
    
    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [71]:
bus1 = HauntedBus(["Raya", "Kiki"])
print(bus1.passengers)
bus1.pick("Dora")
bus1.drop("Raya")
bus1.passengers

['Raya', 'Kiki']


['Kiki', 'Dora']

In [72]:
bus2 = HauntedBus()
bus2.pick("Suri")
print(bus2.passengers)
bus3 = HauntedBus()
bus3.pick("Raya")
bus3.passengers


['Suri']


['Suri', 'Raya']

In [73]:
HauntedBus.__init__.__defaults__

(['Suri', 'Raya'],)

In [77]:
HauntedBus.__init__.__defaults__[0] is bus2.passengers

True

### Defensive Programming with Mutable Parameters 

In [79]:
class TwilightBus:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers  # still it copies the passed argument reference
    
    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [ ]:
cat_team = ["Flow", "Shiro", "Niko", "Ginger", "Chika"]
bus = TwilightBus(cat_team)

bus.drop("Niko")
bus.drop("Chika")

cat_team    # Original list cat_team is modified

['Flow', 'Shiro', 'Ginger']

In [81]:
#Fix
class TwilightBusFix:
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = passengers[:]  # shallow copy
    
    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [82]:
cat_team = ["Flow", "Shiro", "Niko", "Ginger", "Chika"]
bus = TwilightBusFix(cat_team)

bus.drop("Niko")
bus.drop("Chika")

cat_team    # Original list cat_team is not  modified

['Flow', 'Shiro', 'Niko', 'Ginger', 'Chika']

### del and Garbage Collection

del is statement not function we write del x although del(x) also works

del delete reference not objects. Garbage collector will discar an object from memory as in direct result of del

when the reference count of the object reaches zero then object is immediately destroyed

In [83]:
a = [1, 2]
b = a
del a
b

[1, 2]

In [90]:
### Let use weakref to see it in action

import weakref

s1 = {1, 2, 3}
s2 = s1

def bye():
    print("callback ... like tears in rain")

ender = weakref.finalize(s1, bye)

print(ender.alive)
del s1

print(ender.alive)

s2 = "spam"

print(ender.alive)


True
True
callback ... like tears in rain
False


### Tricks Python plays with Immutables

In [92]:
# for tuble t, t[:] dosent make a shallow copy but it returns reference of the same object

t1 = (1, 2, 3)
t2 = tuple(t1)

t1 is t2

True

In [93]:
t3 = t1[:]
t3 is t1

True

In [94]:
# string literals may create share object
t1 = (1, 2, 3)
t2 = (1, 2, 3)
print(t1 is t2)

s1 = "hello"
s2 = "hello"
print(s1 is s2)

False
True
